## 机器学习课程作业
### 1. 作业要求
1．使用基于梯度下降的标准线性回归进行预测  
   ·比较原始数据和不同特征缩放方法对梯度法的影响  
   ·数据均值归一化，分析不同学习率α和初值对梯度法的影响  
2.使用Lasso回归和岭回归进行预测  
3.使用局部加权线性回归进行预测并分析k值对结果的影响  
4.分析影响鲍鱼年龄的主要因素  
5.比较以上4种回归方法的预测效果


### 第一步读取数据

In [1]:
# 从http://archive.ics.uci.edu/dataset/1/abalone下载数据
# 保存到本地文件abalone.data

import urllib.request
import os
import pandas as pd
import numpy as np
import Regression as rd
from sklearn.model_selection import KFold
from sklearn.model_selection import train_test_split

# 下载数据
url = "http://archive.ics.uci.edu/ml/machine-learning-databases/abalone/abalone.data"
filepath = "abalone.data"
if not os.path.isfile(filepath):
    result = urllib.request.urlretrieve(url, filepath)
    print('downloaded:', result)
else:
    print('Data file already exists.')

# 查看数据
import pandas as pd
df = pd.read_csv(filepath, header=None, prefix='V')
# 修改df的列名
df.columns =[ '性别','长度','直径','高度','整体重量','去壳重量','脏器重量','壳的重量','环数']

Data file already exists.


C:\Users\25435\AppData\Local\Temp\ipykernel_26152\2253136986.py:23: FutureWarning: The prefix argument has been deprecated and will be removed in a future version. Use a list comprehension on the column names in the future.


  df = pd.read_csv(filepath, header=None, prefix='V')


### 进行简单的数据预处理

In [2]:
# 使用pandas将df除了性别列的数据进行标准化生成新的df_norm
df_norm = df.iloc[:,1:8]
df_norm = (df_norm - df_norm.mean())/df_norm.std()
df_norm['性别']=df['性别']
df_norm['环数']=df['环数']
# df_norm.head()

#此处决定是否要用标准化后的数据代替原始数据
df=df_norm

df['年龄']=df['环数']+1.5
df['截距']=1
# 使用OneHot对性别进行编码
sex_onehot =pd.get_dummies(df['性别'],prefix='性别')
df[sex_onehot.columns] = sex_onehot

### 构建训练数据和测试数据

In [3]:
# 筛选特征
y=df['环数']
# 避免多重共线性，删除一列
features=['长度','直径','高度','整体重量','去壳重量','脏器重量','壳的重量','截距','性别_F','性别_I']
X=df[features]

### 10折交叉验证，梯度下降方法

In [27]:
kf = KFold(n_splits=10,shuffle=True)
thetas=[]
errs=[]
ks=[]
test_errs=[]
for train_index, test_index in kf.split(X):
    X_train,X_test=X.iloc[train_index],X.iloc[test_index]
    y_train,y_test=y.iloc[train_index],y.iloc[test_index]
    # 使用线性回归模型
    theta= np.zeros((X_train.shape[1], 1))
    # theta= np.ones((X_train.shape[1], 1))
    # theta= np.random.rand(X_train.shape[1], 1)

    theta,error,k=rd.gradient_descent(X_train,y_train,alpha=0.1,theta=theta,iters=50000)
    thetas.append(theta)
    errs.append(error)
    ks.append(k)
    theta = np.array(theta).reshape(-1)
    test_errs.append(rd.err_func(theta,X_test,y_test))

# 选择最优的err,输出并保留7位小数

theta=thetas[test_errs.index(min(test_errs))]
print('最优theta:',theta)
print('最优k:',ks[test_errs.index(min(test_errs))])
print('最优测试集MES:{:.7f}'.format(min(test_errs)))
print('测试集MES均值:{:.7f}'.format(np.mean(test_errs)))

最优theta: [[ 0.03997159]
 [ 1.02831958]
 [ 0.43509178]
 [ 4.52347796]
 [-4.42166801]
 [-1.27958376]
 [ 1.22213544]
 [10.27776122]
 [-0.08128754]
 [-0.94749172]]
最优k: 11824
最优测试集MES:1.9002403
测试集MES均值:2.4660023


### 10折交叉验证，Lasso回归

In [33]:
Lasso_paras=[10,5,2,1,0.5,0.1,0.05,0.01,0.02,0.005]
exper_res_Lasso=pd.DataFrame(columns=['Lasso参数','最优迭代次数','最优测试集MES','测试集MES均值'])
kf = KFold(n_splits=10,shuffle=True)
for i in range(len(Lasso_paras)):

    thetas=[]
    errs=[]
    ks=[]
    test_errs=[]
    for train_index, test_index in kf.split(X):
        X_train,X_test=X.iloc[train_index],X.iloc[test_index]
        y_train,y_test=y.iloc[train_index],y.iloc[test_index]

        theta,error,k=rd.Lasso_regression(X_train,y_train,Lambda=Lasso_paras[i],iters=50000)
        thetas.append(theta)
        errs.append(error)
        ks.append(k)
        theta = np.array(theta).reshape(-1)
        test_errs.append(rd.err_func(theta,X_test,y_test))

    # 选择最优的err,输出并保留7位小数

    exper_res_Lasso=exper_res_Lasso.append({'Lasso参数':Lasso_paras[i],'最优迭代次数':ks[test_errs.index(min(test_errs))],'最优测试集MES':min(test_errs),'测试集MES均值':np.mean(test_errs)},ignore_index=True)
    # theta=thetas[test_errs.index(min(test_errs))]
    # print('最优theta:',theta)
    # print('最优k:',ks[test_errs.index(min(test_errs))])
    # print('最优测试集MES:{:.7f}'.format(min(test_errs)))
    # print('测试集MES均值:{:.7f}'.format(np.mean(test_errs)))

C:\Users\25435\AppData\Local\Temp\ipykernel_27640\803375557.py:28: FutureWarning: The frame.append method is deprecated and will be removed from pandas in a future version. Use pandas.concat instead.
  exper_res=exper_res.append({'Lasso参数':Lasso_paras[i],'最优迭代次数':ks[test_errs.index(min(test_errs))],'最优测试集MES':min(test_errs),'测试集MES均值':np.mean(test_errs)},ignore_index=True)
C:\Users\25435\AppData\Local\Temp\ipykernel_27640\803375557.py:28: FutureWarning: The frame.append method is deprecated and will be removed from pandas in a future version. Use pandas.concat instead.
  exper_res=exper_res.append({'Lasso参数':Lasso_paras[i],'最优迭代次数':ks[test_errs.index(min(test_errs))],'最优测试集MES':min(test_errs),'测试集MES均值':np.mean(test_errs)},ignore_index=True)
C:\Users\25435\AppData\Local\Temp\ipykernel_27640\803375557.py:28: FutureWarning: The frame.append method is deprecated and will be removed from pandas in a future version. Use pandas.concat instead.
  exper_res=exper_res.append({'Lasso参数':Lasso_pa

In [35]:
exper_res_Lasso.to_csv('exper_res_Lasso.csv',index=False)

### 10折交叉验证，岭回归

In [37]:
Lambda_paras=[10,5,2,1,0.5,0.1,0.05,0.01,0.02,0.005]
exper_res_Ridge=pd.DataFrame(columns=['岭回归参数','最优测试集MES','测试集MES均值'])
kf = KFold(n_splits=10,shuffle=True)
for i in range(len(Lambda_paras)):

    thetas=[]
    errs=[]
    test_errs=[]
    for train_index, test_index in kf.split(X):
        X_train,X_test=X.iloc[train_index],X.iloc[test_index]
        y_train,y_test=y.iloc[train_index],y.iloc[test_index]

        theta,error=rd.Ridge(X_train,y_train,Lambda=Lambda_paras[i])
        thetas.append(theta)
        errs.append(error)
        theta = np.array(theta).reshape(-1)
        test_errs.append(rd.err_func(theta,X_test,y_test))

    # 选择最优的err,输出并保留7位小数

    exper_res_Ridge=exper_res_Ridge.append({'岭回归参数':Lambda_paras[i],'最优测试集MES':min(test_errs),'测试集MES均值':np.mean(test_errs)},ignore_index=True)

exper_res_Ridge.to_csv('exper_res_Ridge.csv',index=False)

C:\Users\25435\AppData\Local\Temp\ipykernel_27640\2947834342.py:21: FutureWarning: The frame.append method is deprecated and will be removed from pandas in a future version. Use pandas.concat instead.
  exper_res_Ridge=exper_res_Ridge.append({'岭回归参数':Lambda_paras[i],'最优测试集MES':min(test_errs),'测试集MES均值':np.mean(test_errs)},ignore_index=True)
C:\Users\25435\AppData\Local\Temp\ipykernel_27640\2947834342.py:21: FutureWarning: The frame.append method is deprecated and will be removed from pandas in a future version. Use pandas.concat instead.
  exper_res_Ridge=exper_res_Ridge.append({'岭回归参数':Lambda_paras[i],'最优测试集MES':min(test_errs),'测试集MES均值':np.mean(test_errs)},ignore_index=True)
C:\Users\25435\AppData\Local\Temp\ipykernel_27640\2947834342.py:21: FutureWarning: The frame.append method is deprecated and will be removed from pandas in a future version. Use pandas.concat instead.
  exper_res_Ridge=exper_res_Ridge.append({'岭回归参数':Lambda_paras[i],'最优测试集MES':min(test_errs),'测试集MES均值':np.mean(te

### 10折交叉验证，加权线性回归

In [9]:
K_paras=[1,0.5,0.1,0.05]
exper_res_LWLR=pd.DataFrame(columns=['LWLR参数','最优测试集MES','测试集MES均值'])
kf = KFold(n_splits=10,shuffle=True)
for i in range(len(K_paras)):
    # 计算不同k值下的误差
    test_errs=[]
    for train_index, test_index in kf.split(X):
        X_train,X_test=X.iloc[train_index],X.iloc[test_index]
        y_train,y_test=y.iloc[train_index],y.iloc[test_index]

        #单独计算每个测试样例的误差
        error_list=[]
        for j in range(len(X_test)):
            a=X_test.iloc[j,:]
            np.mat(a).reshape(1,-1)
            t=rd.LW_linear_Regression(a,X_train,y_train,k=K_paras[i])
            if t==0:
                continue
            t=t[0,0]-y_test.iloc[j]
            error_list.append(t**2)
        
        test_errs.append(np.mean(error_list)/2)
    
    exper_res_LWLR=exper_res_LWLR.append({'LWLR参数':K_paras[i],'最优测试集MES':min(test_errs),'测试集MES均值':np.mean(test_errs)},ignore_index=True)

exper_res_LWLR.to_csv('exper_res_LWLR.csv',index=False)                

矩阵为奇异矩阵，不能求逆


C:\Users\25435\AppData\Local\Temp\ipykernel_26152\3135623623.py:24: FutureWarning: The frame.append method is deprecated and will be removed from pandas in a future version. Use pandas.concat instead.
  exper_res_LWLR=exper_res_LWLR.append({'LWLR参数':K_paras[i],'最优测试集MES':min(test_errs),'测试集MES均值':np.mean(test_errs)},ignore_index=True)


矩阵为奇异矩阵，不能求逆
矩阵为奇异矩阵，不能求逆


C:\Users\25435\AppData\Local\Temp\ipykernel_26152\3135623623.py:24: FutureWarning: The frame.append method is deprecated and will be removed from pandas in a future version. Use pandas.concat instead.
  exper_res_LWLR=exper_res_LWLR.append({'LWLR参数':K_paras[i],'最优测试集MES':min(test_errs),'测试集MES均值':np.mean(test_errs)},ignore_index=True)


矩阵为奇异矩阵，不能求逆
矩阵为奇异矩阵，不能求逆
矩阵为奇异矩阵，不能求逆
矩阵为奇异矩阵，不能求逆
矩阵为奇异矩阵，不能求逆
矩阵为奇异矩阵，不能求逆
矩阵为奇异矩阵，不能求逆
矩阵为奇异矩阵，不能求逆
矩阵为奇异矩阵，不能求逆
矩阵为奇异矩阵，不能求逆
矩阵为奇异矩阵，不能求逆
矩阵为奇异矩阵，不能求逆
矩阵为奇异矩阵，不能求逆
矩阵为奇异矩阵，不能求逆
矩阵为奇异矩阵，不能求逆
矩阵为奇异矩阵，不能求逆
矩阵为奇异矩阵，不能求逆
矩阵为奇异矩阵，不能求逆
矩阵为奇异矩阵，不能求逆
矩阵为奇异矩阵，不能求逆
矩阵为奇异矩阵，不能求逆
矩阵为奇异矩阵，不能求逆
矩阵为奇异矩阵，不能求逆
矩阵为奇异矩阵，不能求逆
矩阵为奇异矩阵，不能求逆
矩阵为奇异矩阵，不能求逆
矩阵为奇异矩阵，不能求逆
矩阵为奇异矩阵，不能求逆
矩阵为奇异矩阵，不能求逆
矩阵为奇异矩阵，不能求逆
矩阵为奇异矩阵，不能求逆
矩阵为奇异矩阵，不能求逆
矩阵为奇异矩阵，不能求逆
矩阵为奇异矩阵，不能求逆
矩阵为奇异矩阵，不能求逆
矩阵为奇异矩阵，不能求逆
矩阵为奇异矩阵，不能求逆
矩阵为奇异矩阵，不能求逆
矩阵为奇异矩阵，不能求逆
矩阵为奇异矩阵，不能求逆
矩阵为奇异矩阵，不能求逆
矩阵为奇异矩阵，不能求逆
矩阵为奇异矩阵，不能求逆
矩阵为奇异矩阵，不能求逆
矩阵为奇异矩阵，不能求逆
矩阵为奇异矩阵，不能求逆
矩阵为奇异矩阵，不能求逆
矩阵为奇异矩阵，不能求逆
矩阵为奇异矩阵，不能求逆
矩阵为奇异矩阵，不能求逆
矩阵为奇异矩阵，不能求逆
矩阵为奇异矩阵，不能求逆
矩阵为奇异矩阵，不能求逆
矩阵为奇异矩阵，不能求逆
矩阵为奇异矩阵，不能求逆
矩阵为奇异矩阵，不能求逆
矩阵为奇异矩阵，不能求逆
矩阵为奇异矩阵，不能求逆
矩阵为奇异矩阵，不能求逆
矩阵为奇异矩阵，不能求逆
矩阵为奇异矩阵，不能求逆
矩阵为奇异矩阵，不能求逆
矩阵为奇异矩阵，不能求逆
矩阵为奇异矩阵，不能求逆


...

C:\Users\25435\AppData\Local\Temp\ipykernel_26152\3135623623.py:24: FutureWarning: The frame.append method is deprecated and will be removed from pandas in a future version. Use pandas.concat instead.
  exper_res_LWLR=exper_res_LWLR.append({'LWLR参数':K_paras[i],'最优测试集MES':min(test_errs),'测试集MES均值':np.mean(test_errs)},ignore_index=True)


矩阵为奇异矩阵，不能求逆
矩阵为奇异矩阵，不能求逆
矩阵为奇异矩阵，不能求逆
矩阵为奇异矩阵，不能求逆
矩阵为奇异矩阵，不能求逆
矩阵为奇异矩阵，不能求逆
矩阵为奇异矩阵，不能求逆
矩阵为奇异矩阵，不能求逆
矩阵为奇异矩阵，不能求逆
矩阵为奇异矩阵，不能求逆
矩阵为奇异矩阵，不能求逆
矩阵为奇异矩阵，不能求逆
矩阵为奇异矩阵，不能求逆
矩阵为奇异矩阵，不能求逆
矩阵为奇异矩阵，不能求逆
矩阵为奇异矩阵，不能求逆
矩阵为奇异矩阵，不能求逆
矩阵为奇异矩阵，不能求逆
矩阵为奇异矩阵，不能求逆
矩阵为奇异矩阵，不能求逆
矩阵为奇异矩阵，不能求逆
矩阵为奇异矩阵，不能求逆
矩阵为奇异矩阵，不能求逆
矩阵为奇异矩阵，不能求逆
矩阵为奇异矩阵，不能求逆
矩阵为奇异矩阵，不能求逆
矩阵为奇异矩阵，不能求逆
矩阵为奇异矩阵，不能求逆
矩阵为奇异矩阵，不能求逆
矩阵为奇异矩阵，不能求逆
矩阵为奇异矩阵，不能求逆
矩阵为奇异矩阵，不能求逆
矩阵为奇异矩阵，不能求逆
矩阵为奇异矩阵，不能求逆
矩阵为奇异矩阵，不能求逆
矩阵为奇异矩阵，不能求逆
矩阵为奇异矩阵，不能求逆
矩阵为奇异矩阵，不能求逆
矩阵为奇异矩阵，不能求逆
矩阵为奇异矩阵，不能求逆
矩阵为奇异矩阵，不能求逆
矩阵为奇异矩阵，不能求逆
矩阵为奇异矩阵，不能求逆
矩阵为奇异矩阵，不能求逆
矩阵为奇异矩阵，不能求逆
矩阵为奇异矩阵，不能求逆
矩阵为奇异矩阵，不能求逆
矩阵为奇异矩阵，不能求逆
矩阵为奇异矩阵，不能求逆
矩阵为奇异矩阵，不能求逆
矩阵为奇异矩阵，不能求逆
矩阵为奇异矩阵，不能求逆
矩阵为奇异矩阵，不能求逆
矩阵为奇异矩阵，不能求逆
矩阵为奇异矩阵，不能求逆
矩阵为奇异矩阵，不能求逆
矩阵为奇异矩阵，不能求逆
矩阵为奇异矩阵，不能求逆
矩阵为奇异矩阵，不能求逆
矩阵为奇异矩阵，不能求逆
矩阵为奇异矩阵，不能求逆
矩阵为奇异矩阵，不能求逆
矩阵为奇异矩阵，不能求逆
矩阵为奇异矩阵，不能求逆
矩阵为奇异矩阵，不能求逆
矩阵为奇异矩阵，不能求逆
矩阵为奇异矩阵，不能求逆
矩阵为奇异矩阵，不能求逆
矩阵为奇异矩阵，不能求逆
矩阵为奇异矩阵，不能求逆
矩阵为奇异矩阵，不能求逆
矩阵为奇异矩阵，不能求逆
矩阵为奇异矩阵，不能求逆
矩阵为奇异矩阵，不能求逆
矩阵为奇异矩阵，不能求逆
矩阵为奇异矩阵，不能求逆
矩阵为奇异矩阵，不能求逆

d:\Anaconda3\lib\site-packages\numpy\core\fromnumeric.py:3432: RuntimeWarning: Mean of empty slice.
  return _methods._mean(a, axis=axis, dtype=dtype,
d:\Anaconda3\lib\site-packages\numpy\core\_methods.py:190: RuntimeWarning: invalid value encountered in double_scalars
  ret = ret.dtype.type(ret / rcount)


矩阵为奇异矩阵，不能求逆
矩阵为奇异矩阵，不能求逆
矩阵为奇异矩阵，不能求逆
矩阵为奇异矩阵，不能求逆
矩阵为奇异矩阵，不能求逆
矩阵为奇异矩阵，不能求逆
矩阵为奇异矩阵，不能求逆
矩阵为奇异矩阵，不能求逆
矩阵为奇异矩阵，不能求逆
矩阵为奇异矩阵，不能求逆
矩阵为奇异矩阵，不能求逆
矩阵为奇异矩阵，不能求逆
矩阵为奇异矩阵，不能求逆
矩阵为奇异矩阵，不能求逆
矩阵为奇异矩阵，不能求逆
矩阵为奇异矩阵，不能求逆
矩阵为奇异矩阵，不能求逆
矩阵为奇异矩阵，不能求逆
矩阵为奇异矩阵，不能求逆
矩阵为奇异矩阵，不能求逆
矩阵为奇异矩阵，不能求逆
矩阵为奇异矩阵，不能求逆
矩阵为奇异矩阵，不能求逆
矩阵为奇异矩阵，不能求逆
矩阵为奇异矩阵，不能求逆
矩阵为奇异矩阵，不能求逆
矩阵为奇异矩阵，不能求逆
矩阵为奇异矩阵，不能求逆
矩阵为奇异矩阵，不能求逆
矩阵为奇异矩阵，不能求逆
矩阵为奇异矩阵，不能求逆
矩阵为奇异矩阵，不能求逆
矩阵为奇异矩阵，不能求逆
矩阵为奇异矩阵，不能求逆
矩阵为奇异矩阵，不能求逆
矩阵为奇异矩阵，不能求逆
矩阵为奇异矩阵，不能求逆
矩阵为奇异矩阵，不能求逆
矩阵为奇异矩阵，不能求逆
矩阵为奇异矩阵，不能求逆
矩阵为奇异矩阵，不能求逆
矩阵为奇异矩阵，不能求逆
矩阵为奇异矩阵，不能求逆
矩阵为奇异矩阵，不能求逆
矩阵为奇异矩阵，不能求逆
矩阵为奇异矩阵，不能求逆
矩阵为奇异矩阵，不能求逆
矩阵为奇异矩阵，不能求逆
矩阵为奇异矩阵，不能求逆
矩阵为奇异矩阵，不能求逆
矩阵为奇异矩阵，不能求逆
矩阵为奇异矩阵，不能求逆
矩阵为奇异矩阵，不能求逆
矩阵为奇异矩阵，不能求逆
矩阵为奇异矩阵，不能求逆
矩阵为奇异矩阵，不能求逆
矩阵为奇异矩阵，不能求逆
矩阵为奇异矩阵，不能求逆
矩阵为奇异矩阵，不能求逆
矩阵为奇异矩阵，不能求逆
矩阵为奇异矩阵，不能求逆
矩阵为奇异矩阵，不能求逆
矩阵为奇异矩阵，不能求逆
矩阵为奇异矩阵，不能求逆
矩阵为奇异矩阵，不能求逆
矩阵为奇异矩阵，不能求逆
矩阵为奇异矩阵，不能求逆
矩阵为奇异矩阵，不能求逆
矩阵为奇异矩阵，不能求逆
矩阵为奇异矩阵，不能求逆
矩阵为奇异矩阵，不能求逆
矩阵为奇异矩阵，不能求逆
矩阵为奇异矩阵，不能求逆
矩阵为奇异矩阵，不能求逆
矩阵为奇异矩阵，不能求逆
矩阵为奇异矩阵，不能求逆
矩阵为奇异矩阵，不能求逆

C:\Users\25435\AppData\Local\Temp\ipykernel_26152\3135623623.py:24: FutureWarning: The frame.append method is deprecated and will be removed from pandas in a future version. Use pandas.concat instead.
  exper_res_LWLR=exper_res_LWLR.append({'LWLR参数':K_paras[i],'最优测试集MES':min(test_errs),'测试集MES均值':np.mean(test_errs)},ignore_index=True)
